<a href="https://colab.research.google.com/github/citizenyves/Advanced_NLP_with_PyTorch/blob/main/Pytorch_BERT_for_Sentence_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [BASIC](#Basic)
- BERT 모델의 hidden state에서 **특정 단어의 embedding을 여러 방식으로 추출 및 생성**할 수 있다.

# [CHALLENGE](#Challenge)
- **cosine similarity 함수를 구현**할 수 있다. 
- **단어들의 유사도**를 cosine similarity로 비교할 수 있다. 

# [ADVANCED](#Advanced)
- 문장 embedding을 구해 **문장 간 유사도**를 구할 수 있다.

### Reference
- [BERT word embedding & sentence embedding tutorial 영문 블로그](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/#33-creating-word-and-sentence-vectors-from-hidden-states)

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import torch
import random

## Basic

### BERT 모델과 토크나이저 로드   
- 두 사람의 대화에서 (단어 및 문장의) embedding을 생성하고자 한다. 아래 대화를 BERT 모델에 입력해 출력값 중 "hidden states"값을 가져오자.
- `Hidden States`는 3차원 텐서를 가지고 있는 list 타입이다. List에는 BERT 모델의 각 layer마다의 hidden state 3차원 텐서를 갖고 있으며 각 텐서는 (batch_size, sequence_length, hidden_size) shape을 가진다. BERT-base 모델은 12 layer를 갖고 있고 이와 별도로 Embedding Layer 1개를 더 갖고 있기 때문에 `len(hidden states)`는 13개가 된다. 
    - batch_size: 학습 시 설정한 배치 사이즈. 또는 BERT 모델에 입력된 문장의 개수
    - sequence_length: 문장의 token의 개수. 
    - hidden size: token의 embedding size 
- Reference
    - [BertTokenizer.tokenize() 함수의 매개변수 설명](https://huggingface.co/transformers/v3.0.2/main_classes/tokenizer.html#transformers.PreTrainedTokenizer.__call__)
    - [BERTModel.forward() 함수의 매개변수 및 리턴 값 설명](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel.forward)

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 2.5 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 38.4 MB/s 
     |████████████████████████████████| 895 kB 43.6 MB/s 
     |████████████████████████████████| 596 kB 49.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer_bert = BertTokenizer.from_pretrained("bert-base-cased")
model_bert = BertModel.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
normal_person = ["what do you do when you have free time?"]
nerd = ["I code. code frees my minds, body and soul."]
normal_person.append("(what a nerd...) coding?")
nerd.append("Yes. coding is the best thing to do in the free time.")

for i in range(len(normal_person)):
    print(f"Normal Person asked: {normal_person[i]}")
    print(f"Nerd answers: {nerd[i]}")

Normal Person asked: what do you do when you have free time?
Nerd answers: I code. code frees my minds, body and soul.
Normal Person asked: (what a nerd...) coding?
Nerd answers: Yes. coding is the best thing to do in the free time.


In [ ]:
# 매개변수 설명
# truncation <- max_len 넘어가지 않도록 자르기
# padding <- max(seq_len, max_len) zero padding 
# padding='longest' or True : pad to the longest sequence in the batch
# return_tensors <- return 2d tensor 

inputs = tokenizer_bert(
    text = normal_person, 
    text_pair = nerd, #BERT는 두개의 문장까지 입력 가능
    truncation = True, 
    padding = "longest", 
    #return_special_tokens_mask = True,
    return_tensors='pt' #'pt':torch.Tensor | 'tf':tf.constant | 'np':Numpy
    )

print(inputs['input_ids'].shape)
print("\n")
print(inputs['input_ids']) #101:[CLS] | 102:[SEP] | 0:padded
print(inputs['token_type_ids']) # segment of sentences
print(inputs['attention_mask']) #0 if padded, and 1 for one not padded
#print(inputs['special_tokens_mask']) #1 if special token or padded, and 0 for the rests

torch.Size([2, 28])


tensor([[  101,  1184,  1202,  1128,  1202,  1165,  1128,  1138,  1714,  1159,
           136,   102,   146,  3463,   119,  3463,  1714,  1116,  1139, 10089,
           117,  1404,  1105,  3960,   119,   102,     0,     0],
        [  101,   113,  1184,   170, 24928,  2956,   119,   119,   119,   114,
         19350,   136,   102,  2160,   119, 19350,  1110,  1103,  1436,  1645,
          1106,  1202,  1107,  1103,  1714,  1159,   119,   102]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])


In [ ]:
# decoding
for i in range(len(inputs['input_ids'])):
    print(f"Coversation {i} -> '{tokenizer_bert.decode(inputs['input_ids'][i])}'")

Coversation 0 -> '[CLS] what do you do when you have free time? [SEP] I code. code frees my minds, body and soul. [SEP] [PAD] [PAD]'
Coversation 1 -> '[CLS] ( what a nerd... ) coding? [SEP] Yes. coding is the best thing to do in the free time. [SEP]'


In [ ]:
# .encode()
# "code" 단어의 token id(각 단어에게 고유하게 주어진 id)를 출력
tokenizer_bert.encode('code', add_special_tokens=False)

[3463]

In [ ]:
# assigning cuda or cpu
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


In [ ]:
# 입력 데이터와 BERT 모델을 "GPU" 장치로 로드함
inputs = inputs.to(device)
model_bert.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
# 입력 데이터를 BERT 모델에 넣어 출력값을 가져옴
outputs = model_bert(
    **inputs, 
    output_hidden_states=True
    )

In [ ]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])

In [ ]:
# hidden_states has four dimensions, in the following order: The layer number, The batch number, The word/token number, The hidden unit/feature number(768 features)
hidden_states = outputs['hidden_states']
print(f"Number of layers : {len(hidden_states)}") #13, including the input embeddings as the first element
print(f"tensor shape in each layer : {hidden_states[-1].shape}")
print('\n')
layer_i = 0
print(f"Number of batches : {len(hidden_states[layer_i])}")
batch_i = 0
print(f"Number of tokens : {len(hidden_states[layer_i][batch_i])}")
token_i = 0
print(f"Number of hidden units : {len(hidden_states[layer_i][batch_i][token_i])}")

Number of layers : 13
tensor shape in each layer : torch.Size([2, 28, 768])


Number of batches : 2
Number of tokens : 28
Number of hidden units : 768


###  Q1. 1번째 sequence (문장)에서 "code"라는 단어의 인덱스를 모두 반환하라.
- "code" 단어는 총 2개 존재 

In [ ]:
# 인덱스 뽑는 다른방법 참고
# (seq == tokenizer_bert.convert_tokens_to_ids(word)).nonzero(as_tuple=True)[0]

In [ ]:
def get_index(seq, word):
    index = [idx for idx, ids in enumerate(seq) if ids == tokenizer_bert.convert_tokens_to_ids(word)]
    return index

# input
# seq1: 1번째 sequence
# token: 단어
seq1 = inputs['input_ids'][0]
token = "code"

# output
token_index = get_index(seq1, token)
print(token_index)

[13, 15]


### Q2. 1번째 sequence의 1번째 "code" 토큰의 embedding을 여러가지 방식으로 구하고자 한다. BERT hidden state를 다음의 방식으로 인덱싱해 embedding을 구하라
- 1 layer
- last layer
- sum all 12 layers
- sum last 4 layers
- concat last 4 layers
- average last 4 layers

In [ ]:
# 첫번째 'code' 인덱스 / 두번째 'code' 인덱스 
idx_first_code = token_index[0] # 13
idx_second_code = token_index[1] # 15
print(f"index number of the first 'code'                   : {idx_first_code}")
print(f"index number of the second 'code'                  : {idx_second_code}")

# Combine the layers to make this one whole big tensor
token_embeddings = torch.stack(hidden_states, dim=0) #레이어
print(f"token embeddings                                   : {token_embeddings.size()}")

# 1번째 sequence 임베딩
first_seq_token_embeddings = token_embeddings[:, 0]
print(f"token embeddings of first sequence                 : {first_seq_token_embeddings.size()}")

# Switch around the "layers" and "tokens" dimensions with permute
first_seq_token_embeddings = first_seq_token_embeddings.permute(1,0,2)
print(f"token embeddings of first sequence after switching : {first_seq_token_embeddings.size()}")

index number of the first 'code'                   : 13
index number of the second 'code'                  : 15
token embeddings                                   : torch.Size([13, 2, 28, 768])
token embeddings of first sequence                 : torch.Size([13, 28, 768])
token embeddings of first sequence after switching : torch.Size([28, 13, 768])


In [ ]:
# # 1 layer
first_layer_emb = first_seq_token_embeddings[idx_first_code][1] #첫번째 'code' 인덱스 + 첫번째 레이어(인풋 임베딩 레이어 제외)
print(first_layer_emb.shape)

# # last layer
last_layer_emb = first_seq_token_embeddings[idx_first_code][-1] # 첫번째 'code' 인덱스 + 마지막 레이어
print(last_layer_emb.shape)

# # sum all 12 layers
sum_all_layer_emb = [torch.sum(token[1:], dim=0) #인풋 임베딩 레이어 제외 : token[1:]
                     for token in first_seq_token_embeddings][idx_first_code] #첫번째 'code' 인덱스
print(sum_all_layer_emb.shape)

# # sum last 4 layers
sum_last4_layer_emb = [torch.sum(token[-4:], dim=0) 
                       for token in first_seq_token_embeddings][idx_first_code] #첫번째 'code' 인덱스
print(sum_last4_layer_emb.shape)

# concat last 4 layers
concat_last4_layer_emb = [torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0) 
                          for token in first_seq_token_embeddings][idx_first_code] #첫번째 'code' 인덱스
print(concat_last4_layer_emb.shape)

# # mean last 4 layers
mean_last4_layer_emb = [torch.mean(token[-4:], dim=0) 
                        for token in first_seq_token_embeddings][idx_first_code] #첫번째 'code' 인덱스
print(mean_last4_layer_emb.shape)

torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([3072])
torch.Size([768])


## Challenge

### Q3. `sum_last_four_layer` 방식으로 1번째 sequence의 2개의 "code" 토큰 사이의 코사인 유사도를 계산하라

In [ ]:
# 두번째 'code' 임베딩 : sum_last_four_layer
concat_last4_layer_emb_second = [torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0) 
                                 for token in first_seq_token_embeddings][idx_second_code] #두번째 'code' 인덱스
print(concat_last4_layer_emb_second.shape)

torch.Size([3072])


In [ ]:
def cosine_similarity_manual(x, y, small_number=1e-8):
    # gpu -> cpu -> numpy
    x = x.detach().cpu().numpy()
    y = y.detach().cpu().numpy()

    # cosine similarity 
    cos = np.dot(x, y) / ((np.linalg.norm(x) * (np.linalg.norm(y))) + small_number)

    # numpy -> tensor -> gpu
    result = torch.as_tensor(np.array(cos).astype('float')).to(device)

    return result

# input
# x: 1번째 sequence의 1번째 "code"의 sum_last_four_layer 방식 embedding
# y: 1번째 sequence의 2번째 "code"의 sum_last_four_layer 방식 embedding
x = concat_last4_layer_emb
y = concat_last4_layer_emb_second

# output
score = cosine_similarity_manual(x, y)
print(score)

tensor(0.8336, device='cuda:0', dtype=torch.float64)


### Q4. 2번째 sequence에서 "coding"이라는 토큰의 위치를 반환하라

In [ ]:
# Q1과 동일한 문제 

# input
# seq1: 2번째 sequence
# token: 단어
seq2 = inputs['input_ids'][1]
token = "coding"

# output
# Q1에서 구현한 함수 사용
token_index = get_index(seq2, token)
print(token_index)

[10, 15]


### Q5. `concat_last4_layer_emb` 방식으로 2번째 sequence의 2개의 "coding" 토큰 사이의 코사인 유사도를 계산하라

In [ ]:
# Q3과 동일한 문제
idx_first_coding = token_index[0] # 10
idx_second_coding = token_index[1] # 15

# 2번째 sequence 임베딩
second_seq_token_embeddings = token_embeddings[:, 1]

# Switch around the "layers" and "tokens" dimensions with permute
second_seq_token_embeddings = second_seq_token_embeddings.permute(1,0,2)
print(f"token embeddings of first sequence after switching.         : {first_seq_token_embeddings.size()}")


concat_last4_layer_emb = [torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0) 
                          for token in second_seq_token_embeddings][idx_first_coding] #첫번째 'coding' 인덱스
print(f"embbeding of first 'coding' concatenated with last4 layers  : {concat_last4_layer_emb.shape}")

concat_last4_layer_emb_second = [torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0) 
                                 for token in second_seq_token_embeddings][idx_second_coding] #두번째 'coding' 인덱스
print(f"embbeding of second 'coding' concatenated with last4 layers : {concat_last4_layer_emb_second.shape}")

# input
# x: 2번째 sequence의 1번째 "coding"의 concat_last4_layer_emb
# y: 2번째 sequence의 2번째 "coding"의 concat_last4_layer_emb
x = concat_last4_layer_emb
y = concat_last4_layer_emb_second

# output
# Q3에서 구현한 함수 사용
score = cosine_similarity_manual(x, y)
print(f"cosine similarity                                           : {score}")

token embeddings of first sequence after switching.         : torch.Size([28, 13, 768])
embbeding of first 'coding' concatenated with last4 layers  : torch.Size([3072])
embbeding of second 'coding' concatenated with last4 layers : torch.Size([3072])
cosine similarity                                           : 0.8681786268598455


### Q6. 2번째 sequence에서 랜덤하게 토큰 하나를 뽑아보자. 그 랜덤 토큰과 2번째 sequence의 2번째 "coding" 토큰의 코사인 유사도를 계산해보자

In [ ]:
# input
# random_idx: random 모듈 사용하여 뽑은 랜덤 토큰의 인덱스
# random_word: random_idx에 해당하는 단어
# x: 2번째 sequence의 2번째 "coding" 토큰의 concat_last4_layer_emb
# y: 랜덤 토큰의 concat_last4_layer_emb

# 2번째 sequence에서 random token 뽑기
seq2 = inputs['input_ids'][1].tolist() #2번째 sequence 텐서

random_idx = random.choice(seq2) # 랜덤 토큰 id
print(f"id of a random token   : {random_idx}")
random_word = tokenizer_bert.convert_ids_to_tokens(random_idx) # id에 대한 word 
print(f"a random word          : {random_word}")

random_token_index = get_index(seq2, random_word) #랜덤 토큰 index
print(f"index of a random word : {random_token_index[0]}")

# random token - concat last 4 layers
concat_last4_layer_emb_random = [torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0) 
                                 for token in second_seq_token_embeddings][random_token_index[0]] #random token 인덱스

x = concat_last4_layer_emb_second #2번째 sequence의 2번째 "coding" 토큰의 concat_last4_layer_emb
y = concat_last4_layer_emb_random #2번째 sequence의 랜덤 토큰의 concat_last4_layer_emb

# output
# Q3에서 구현한 함수 사용
score = cosine_similarity_manual(x, y)
print("\n")
print(f"cosine similarity      : {score}")

id of a random token   : 1159
a random word          : time
index of a random word : 25


cosine similarity      : 0.6280659348251697


## Advanced

### Q7. 1번째 sequence와 2번째 sequence의 문장 유사도를 구해보자. 문장의 엠베딩은 마지막 레이어의 첫번째 토큰 ('[CLS]')으로 생성한다.

In [ ]:
# input
# x: 1번째 sequence의 embedding
# y: 2번째 sequence의 embedding
x = outputs['pooler_output'][0]
y = outputs['pooler_output'][1]

# output
# Q3에서 구현한 함수 사용
score = cosine_similarity_manual(x, y)
print(f"cosine similarity : {score}")

cosine similarity : 0.14665922285629518
